In [ ]:
# import libraries (you may add additional imports but you may not have to)
import numpy as np
import pandas as pd
from scipy.sparse import csr_matrix
from sklearn.neighbors import NearestNeighbors
import matplotlib.pyplot as plt

In [ ]:
# get data files
!wget https://cdn.freecodecamp.org/project-data/books/book-crossings.zip

!unzip book-crossings.zip

books_filename = 'BX-Books.csv'
ratings_filename = 'BX-Book-Ratings.csv'

In [ ]:
# import csv data into dataframes
df_books = pd.read_csv(
    books_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['isbn', 'title', 'author'],
    usecols=['isbn', 'title', 'author'],
    dtype={'isbn': 'str', 'title': 'str', 'author': 'str'})

df_ratings = pd.read_csv(
    ratings_filename,
    encoding = "ISO-8859-1",
    sep=";",
    header=0,
    names=['user', 'isbn', 'rating'],
    usecols=['user', 'isbn', 'rating'],
    dtype={'user': 'int32', 'isbn': 'str', 'rating': 'float32'})

In [ ]:
# Get users with ≥200 ratings (from full dataset)
user_count = df_ratings['user'].value_counts()
filtered_users = user_count[user_count >= 200]

# Get books with ≥100 ratings (from full dataset)
book_count = df_ratings['isbn'].value_counts()
filtered_books = book_count[book_count >= 100]

# Apply both filters to get final dataset
filtered_ratings = df_ratings[
    (df_ratings['user'].isin(filtered_users.index)) &
    (df_ratings['isbn'].isin(filtered_books.index))
]

filtered_ratings

In [ ]:
ratings_with_titles = filtered_ratings.merge(df_books, on='isbn')

user_book_matrix = ratings_with_titles.pivot_table(index='title', columns='user', values='rating').fillna(0)

model = NearestNeighbors(metric='cosine', algorithm='brute')
model.fit(user_book_matrix.values)
book_to_index = pd.Series(data=range(len(user_book_matrix)), index=user_book_matrix.index)

In [ ]:
# function to return recommended books - this will be tested
def get_recommends(book_title):
    # Check if title exists
    if book_title not in book_to_index:
        return [book_title, []]

    book_idx = book_to_index[book_title]
    book_vector = user_book_matrix.iloc[book_idx].values.reshape(1, -1)

    # Get top 6 (first one is itself)
    distances, indices = model.kneighbors(book_vector, n_neighbors=6)

    # Format output
    recommendations = []
    for i in range(1, len(indices[0])):
        similar_book = user_book_matrix.index[indices[0][i]]
        distance = float(distances[0][i])
        recommendations.append([similar_book, distance])

    return [book_title, recommendations]


In [ ]:
books = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
print(books)

def test_book_recommendation():
  test_pass = True
  recommends = get_recommends("Where the Heart Is (Oprah's Book Club (Paperback))")
  if recommends[0] != "Where the Heart Is (Oprah's Book Club (Paperback))":
    test_pass = False
  recommended_books = ["I'll Be Seeing You", 'The Weight of Water', 'The Surgeon', 'I Know This Much Is True']
  recommended_books_dist = [0.8, 0.77, 0.77, 0.77]
  for i in range(2):
    if recommends[1][i][0] not in recommended_books:
      test_pass = False
    if abs(recommends[1][i][1] - recommended_books_dist[i]) >= 0.05:
      test_pass = False
  if test_pass:
    print("You passed the challenge! 🎉🎉🎉🎉🎉")
  else:
    print("You haven't passed yet. Keep trying!")

test_book_recommendation()